In [1]:
import pandas as pd
import numpy as np

ZMM = pd.read_csv("Files/5. Emisiones por Zona Metropolitana.csv", encoding = "iso-8859-1")
ZMM[ZMM[" Nombre Zona Metropolitana "] == "Monterrey"]

,Clave Zona Metropolitana,Nombre Zona Metropolitana,PM10,PM2.5,SO2,CO,NOX,COV,NH3
41,19.01,Monterrey,"14,157","9,091","11,809","342,779","122,766","157,911","10,595"


In [2]:
Municipios = pd.read_csv("Files/4. Emisiones por Municipio.csv", encoding = "iso-8859-1")
municipios_ZMM = [
    "Apodaca",
    "Cadereyta Jiménez",
    "El Carmen",
    "Ciénega de Flores",
    "García",
    "San Pedro Garza García",
    "General Escobedo",
    "General Zuazua",
    "Guadalupe",
    "Juárez",
    "Monterrey",
    "Pesquería",
    "Salinas Victoria",
    "San Nicolás de los Garza",
    "Santa Catarina",
    "Santiago"
]

# Aplicando el filtro correctamente
filtro = (Municipios["Estado"] == "Nuevo León") & (Municipios["Municipio"].isin(municipios_ZMM))
Municipios = Municipios[filtro]

# Limpiar y convertir las columnas relevantes a float
def convertir_a_float(col):
    if col.dtype == "object":
        col = col.str.replace(",", "", regex=True)  # Eliminar comas
        col = pd.to_numeric(col, errors="coerce")  # Convertir a float, manejando errores
    return col

# Reemplazar las columnas originales usando assign
Municipios = Municipios.assign(**{
    col: convertir_a_float(Municipios[col]) for col in Municipios.iloc[:, 3:].columns
})

# Confirmar el tipo de datos después de la conversión
print(Municipios.dtypes)


Estado              object
Clave municipal    float64
Municipio           object
  PM10             float64
  PM2.5            float64
  SO2              float64
  CO               float64
  NOX              float64
  COV              float64
  NH3              float64
dtype: object


In [3]:
contaminantes = Municipios.iloc[:, 3:].max(axis=0)

lista = []
for index in Municipios.iloc[:, 3:].idxmax(axis=0).values:
    lista.append(Municipios["Municipio"].filter(items=[index], axis=0).values[0])

tabla = pd.DataFrame({"Contaminante": contaminantes.index, 
                      "Valor máximo": contaminantes.values, 
                      "Porcentaje de presencia del contaminante": np.round(contaminantes.values * 100 / Municipios.iloc[:, 3:].sum(axis = 0).values, 2), 
                      "Municipio correspondiente": lista})
tabla

,Contaminante,Valor máximo,Porcentaje de presencia del contaminante,Municipio correspondiente
0,PM10,2400.41,16.98,Monterrey
1,PM2.5,1614.15,17.78,Monterrey
2,SO2,8736.24,73.98,Cadereyta Jiménez
3,CO,119590.94,34.99,Monterrey
4,NOX,38001.98,31.08,Monterrey
5,COV,38909.20,24.78,Monterrey
6,NH3,2118.66,20.22,Pesquería
